In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! cp "/content/drive/MyDrive/垃圾分類/sample.zip" /content/

In [ ]:
! unzip /content/sample.zip > data_unzip.log

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from keras.applications.resnet import ResNet50
#from tensorflow.python.keras.applications.resnet50 import ResNet50
from tensorflow.keras.optimizers import Adam
#from tensorflow.python.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

def save_model(model, save_name="mymodel"):
    """
    model : Keras Model class
    save_name : str
    """
    # 儲存模型-tf格式
    # Convert the model.
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    # Save the model.
    with open("./" + save_name + ".tflite", 'wb') as f:
        f.write(tflite_model)

    # 儲存模型-keras格式
    model.save("./" + save_name + ".h5")

# 資料路徑
DATASET_PATH  = 'sample'

# 影像大小
IMAGE_SIZE = (224, 224)

# 影像類別數
NUM_CLASSES = 6

# 若 GPU 記憶體不足，可調降 batch size 或凍結更多層網路
BATCH_SIZE = 8

# 凍結網路層數
FREEZE_LAYERS = 2

# Epoch 數
NUM_EPOCHS = 10

# 模型輸出儲存的檔案
WEIGHTS_FINAL = 'model-resnet50-final.h5'

# 透過 data augmentation 產生訓練與驗證用的影像資料
train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_batches = train_datagen.flow_from_directory(DATASET_PATH + '/train',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)

valid_datagen = ImageDataGenerator()
valid_batches = valid_datagen.flow_from_directory(DATASET_PATH + '/valid',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=BATCH_SIZE)

# 輸出各類別的索引值
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))

# 以訓練好的 ResNet50 為基礎來建立模型，
# 捨棄 ResNet50 頂層的 fully connected layers
net = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
x = net.output
x = Flatten()(x)

# 增加 DropOut layer
x = Dropout(0.5)(x)

# 增加 Dense layer，以 softmax 產生個類別的機率值
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)

# 設定凍結與要進行訓練的網路層
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True

# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])

# 輸出整個網路結構
print(net_final.summary())

# 訓練模型
train_model=net_final.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS)

# 儲存訓練好的模型
net_final.save(WEIGHTS_FINAL)

acc = train_model.history['accuracy']
val_acc = train_model.history['val_accuracy']

loss = train_model.history['loss']
val_loss = train_model.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)

plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

test_loss, test_acc = net_final.evaluate_generator(valid_batches,
                            steps=valid_batches.samples//BATCH_SIZE,
                            verbose=1)
print('test acc:', test_acc)
print('test loss:', test_loss)

Found 8496 images belonging to 6 classes.
Found 2124 images belonging to 6 classes.
Class #0 = battery
Class #1 = clothes
Class #2 = metal
Class #3 = paper
Class #4 = plastic
Class #5 = white-glass
94781440/94765736 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)         

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


                                                                                                  
 conv2_block3_3_bn (BatchNormal  (None, 56, 56, 256)  1024       ['conv2_block3_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_add (Add)         (None, 56, 56, 256)  0           ['conv2_block2_out[0][0]',       
                                                                  'conv2_block3_3_bn[0][0]']      
                                                                                                  
 conv2_block3_out (Activation)  (None, 56, 56, 256)  0           ['conv2_block3_add[0][0]']       
                                                                                                  
 conv3_block1_1_conv (Conv2D)   (None, 28, 28, 128)  32896       ['conv2_block3_out[0][0]']       
          

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:109: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
1062/1062 [==============================] - 212s 183ms/step - loss: 1.2259 - accuracy: 0.6879 - val_loss: 0.4556 - val_accuracy: 0.8547
Epoch 2/10
1062/1062 [==============================] - 191s 180ms/step - loss: 0.5958 - accuracy: 0.8293 - val_loss: 0.3161 - val_accuracy: 0.8991
Epoch 3/10
1062/1062 [==============================] - 192s 181ms/step - loss: 0.4295 - accuracy: 0.8759 - val_loss: 0.2898 - val_accuracy: 0.9104
Epoch 4/10
1062/1062 [==============================] - 192s 181ms/step - loss: 0.3356 - accuracy: 0.9008 - val_loss: 0.2282 - val_accuracy: 0.9264
Epoch 5/10
1062/1062 [==============================] - 191s 179ms/step - loss: 0.2951 - accuracy: 0.9127 - val_loss: 0.1875 - val_accuracy: 0.9420
Epoch 6/10
1062/1062 [==============================] - 190s 179ms/step - loss: 0.2559 - accuracy: 0.9213 - val_loss: 0.1869 - val_accuracy: 0.9434
Epoch 7/10
1062/1062 [==============================] - 189s 178ms/step - loss: 0.2207 - accuracy: 0.9306 - val_

NameError: ignored

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
#Confution Matrix and Classification Report
Y_pred = net_final.predict_generator(valid_batches, valid_batches.samples // BATCH_SIZE+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(valid_batches.classes, y_pred))
print('Classification Report')
target_names = ['battery', 'clothes', 'metal', 'paper', 'plastic', 'white-glass']
print(classification_report(valid_batches.classes, y_pred, target_names=target_names))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  after removing the cwd from sys.path.


Confusion Matrix
[[ 182    0    4    1    2    0]
 [   0 1060    0    4    1    0]
 [   1    0  143    3    5    2]
 [   3    1    2  378    3    1]
 [   1    0    5    2  159    6]
 [   0    0   14    2   21  118]]
Classification Report
              precision    recall  f1-score   support

     battery       0.97      0.96      0.97       189
     clothes       1.00      1.00      1.00      1065
       metal       0.85      0.93      0.89       154
       paper       0.97      0.97      0.97       388
     plastic       0.83      0.92      0.87       173
 white-glass       0.93      0.76      0.84       155

    accuracy                           0.96      2124
   macro avg       0.93      0.92      0.92      2124
weighted avg       0.96      0.96      0.96      2124

